In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
Rstats = importr('stats')

In [2]:
counts = pd.read_csv('nature-alddist-counts.csv', index_col=0)
counts = counts[['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0']]
counts = counts.transpose()
counts = counts.drop(['All'], axis=1)
counts.head()

Nature,BUSINESS CHECK,TRAFFIC STOP,RETURN STATION,TRBL W/SUBJ,PATROL,FOLLOW UP,INVESTIGATION,REPORTS,WELFARE CITIZEN,ACC PDO,...,BATTERY,THEFT,ENTRY,ASSIGNMENT,SUBJ WANTED,OUT OF SERVICE,BATTERY DV,INJ PERSON/SICK,TRAFFIC HAZARD,SHOTSPOTTER
0.0,1048,580,7,431,1045,223,170,0,225,196,...,185,171,124,47,103,3,134,81,184,0
1.0,1790,1107,974,772,801,294,367,70,240,224,...,212,219,142,238,138,11,164,127,128,91
2.0,3175,380,18,361,351,122,87,1,168,153,...,87,129,63,34,41,4,58,44,94,0
3.0,2113,786,3122,538,603,451,450,1164,215,266,...,178,160,110,409,172,692,122,110,170,2
4.0,940,1667,2547,734,144,633,539,1181,264,395,...,298,167,176,500,207,711,205,173,196,162


In [3]:
# convert the table to percents out of 100
pcents = counts.copy()
for idx, row in pcents.iterrows():
    total = row.sum()
    for col in pcents.columns:
        pcents.loc[idx, col] = 100*pcents.loc[idx, col] / total
pcents

Nature,BUSINESS CHECK,TRAFFIC STOP,RETURN STATION,TRBL W/SUBJ,PATROL,FOLLOW UP,INVESTIGATION,REPORTS,WELFARE CITIZEN,ACC PDO,...,BATTERY,THEFT,ENTRY,ASSIGNMENT,SUBJ WANTED,OUT OF SERVICE,BATTERY DV,INJ PERSON/SICK,TRAFFIC HAZARD,SHOTSPOTTER
0.0,19.187111,10.618821,0.128158,7.890882,19.132186,4.082754,3.112413,0.000000,4.119370,3.588429,...,3.387038,3.130721,2.270231,0.860491,1.885756,0.054925,2.453314,1.482973,3.368729,0.000000
1.0,18.282096,11.306302,9.947911,7.884792,8.180983,3.002758,3.748340,0.714942,2.451231,2.287815,...,2.165254,2.236748,1.450312,2.430804,1.409458,0.112348,1.675008,1.297110,1.307323,0.929425
2.0,54.600172,6.534824,0.309544,6.208083,6.036113,2.098022,1.496131,0.017197,2.889080,2.631126,...,1.496131,2.218401,1.083405,0.584695,0.705073,0.068788,0.997420,0.756664,1.616509,0.000000
3.0,15.918337,5.921350,23.519662,4.053036,4.542715,3.397619,3.390086,8.769022,1.619708,2.003917,...,1.340967,1.205364,0.828688,3.081211,1.295766,5.213199,0.919090,0.828688,1.280699,0.015067
4.0,7.012309,12.435658,19.000373,5.475569,1.074226,4.722119,4.020888,8.810145,1.969414,2.946662,...,2.223051,1.245804,1.312943,3.729952,1.544200,5.303991,1.529280,1.290563,1.462141,1.208504
5.0,17.946296,15.301579,2.374848,12.454460,1.808123,4.425853,4.952098,1.511267,4.614762,3.764674,...,3.251923,4.021050,2.118473,0.985022,2.523276,0.701660,2.739172,1.889084,1.619215,0.985022
6.0,26.605364,4.321839,13.256705,3.708812,3.724138,5.900383,2.084291,6.881226,1.977011,2.360153,...,1.440613,1.164751,0.750958,2.467433,0.812261,0.183908,0.750958,0.796935,1.486590,0.061303
7.0,46.094682,8.871502,0.344782,10.104761,4.760642,3.368254,1.471953,0.026522,3.408036,3.063254,...,1.750431,3.301949,1.153693,0.676303,0.371304,0.000000,0.318260,1.604562,0.994563,0.000000
8.0,8.885475,5.185327,19.595416,5.313360,2.112541,4.244287,6.420844,7.470713,1.709238,2.246975,...,2.144549,1.241918,1.280328,3.200819,1.440369,4.263491,1.235516,1.126688,1.075475,2.118942
9.0,12.676056,5.787852,9.441021,10.854974,1.744058,6.040933,3.609155,5.122139,2.998460,2.360255,...,2.250220,2.030150,0.764745,4.406910,1.804577,3.741197,1.001320,1.903609,1.028829,0.225572


In [4]:
# run a chi squared test on the entire table
chi2 = stats.chi2_contingency(pcents)
chi2[1]
# below the critical p threshold of alpha / # of comparisons

1.3799894566926134e-15

In [5]:
# confirming the correct amount of data
chi2[3].shape

(15, 25)

In [6]:
# confirm all cells are >= 5
expected = chi2[3]
len(expected[expected < 5])

300

In [7]:
# tons of values are < 5, must do a fisher's exact test
# this needs to be done in R
fisher = np.asarray(Rstats.fisher_test(pcents.values, simulate_p_value=True))
fisher[0][0]

'0.0004997501249375312'

In [8]:
# run a one-way chi squared on each district
pvals = {}
for idx, row in pcents.iterrows():
    pvals[idx] = stats.chisquare(row).pvalue
pvals

{'0.0': 6.020965388700168e-22,
 '1.0': 6.47653522584613e-13,
 '2.0': 1.1056024700304029e-129,
 '3.0': 1.6848763665589152e-23,
 '4.0': 1.690485768109936e-12,
 '5.0': 1.1265617286381736e-14,
 '6.0': 4.998086822442837e-27,
 '7.0': 1.3290819635597776e-90,
 '8.0': 4.393449769001772e-11,
 '9.0': 6.994536779706752e-06,
 '10.0': 7.665048528327615e-10,
 '11.0': 2.644790900474199e-41,
 '12.0': 4.735708489963766e-29,
 '13.0': 1.858734342564123e-38,
 '14.0': 8.22726835511384e-26}